In [1]:
! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 35.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.1 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm
import tensorflow_addons as tfa

In [3]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return cv2.resize(img[35:-35,35:-35], (160,160))

In [4]:
def window(img):
    img[np.where(img<0)] = 0
    img[np.where(img>1900)] = 1900
    return img/1900.0

In [5]:
def channeling(img):
    std = img.std()
    mean = img.mean()
    img1 = img.copy()
    img2 = img.copy()
    
    img1[np.where(img1<(mean-std))] = (mean-std)
    img1[np.where(img1>(mean))] = mean
    
    img2[np.where(img2<(mean))] = mean
    img2[np.where(img2>(mean+std))] = (mean+std)
    
    img[np.where(img<(mean+std))]= (mean+std)
    
    return np.stack([img, img2, img1], axis=-1)

In [6]:
bb_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
bb_df

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137
...,...,...,...,...,...,...
7212,1.2.826.0.1.3680043.9940,297.23186,115.53983,85.18228,66.52623,140
7213,1.2.826.0.1.3680043.9940,298.00000,117.00000,86.00000,61.00000,141
7214,1.2.826.0.1.3680043.9940,298.00000,119.00000,87.00000,58.00000,142
7215,1.2.826.0.1.3680043.9940,299.00000,120.00000,89.00000,56.00000,143


In [7]:
df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
df = df[df['patient_overall']==0]
df

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
7,1.2.826.0.1.3680043.17364,0,0,0,0,0,0,0,0
8,1.2.826.0.1.3680043.849,0,0,0,0,0,0,0,0
12,1.2.826.0.1.3680043.24946,0,0,0,0,0,0,0,0
13,1.2.826.0.1.3680043.9290,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2011,1.2.826.0.1.3680043.32263,0,0,0,0,0,0,0,0
2012,1.2.826.0.1.3680043.6398,0,0,0,0,0,0,0,0
2013,1.2.826.0.1.3680043.31621,0,0,0,0,0,0,0,0
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0


In [8]:
path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/"

In [9]:
import random

def my_distribution(maxx):
    THRESHOLD = [0.15, 0.85]
    u = random.random()
    if u <= THRESHOLD[0]:
        return random.randint(1, maxx//5)
    elif u <= THRESHOLD[1]:
        return random.randint( (maxx//5),  ((4*maxx)//5))
    else:
        return random.randint(((4*maxx)//5), maxx)

In [16]:
np_list = np.arange(0,7217*2)
random_list = np.random.choice(np_list,7217*2,replace=False)
def data_gen():
    for i in random_list[:13000]:
        if i%2==0:
            try:
                label = 0
                img = window(load_image(path+df.iloc[i//2]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[i//2]["StudyInstanceUID"]))))+".dcm"))
                yield channeling(img),label
            except:
                continue

        else:         
            label = 1
            x = path+bb_df.iloc[i//2]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//2]["slice_number"])+".dcm"
            img = window(load_image(path+bb_df.iloc[i//2]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//2]["slice_number"])+".dcm"))
            yield channeling(img),label
        
        
def test_gen():
    for i in random_list[13000:]:
        if i%2==0:
            try:
                label = 0
                img = window(load_image(path+df.iloc[i//2]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[i//2]["StudyInstanceUID"]))))+".dcm"))
                yield channeling(img),label
            except:
                continue

        else:         
            label = 1
            x = path+bb_df.iloc[i//2]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//2]["slice_number"])+".dcm"
            img = window(load_image(path+bb_df.iloc[i//2]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//2]["slice_number"])+".dcm"))
            yield channeling(img),label



In [17]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
    (tf.float32, tf.int8),
    (tf.TensorShape([160,160,3]), tf.TensorShape([])))
testset = tf.data.Dataset.from_generator(
     test_gen,
    (tf.float32, tf.int8),
    (tf.TensorShape([160,160,3]), tf.TensorShape([])))

In [18]:
p=0
n=0
for i in dataset:
    if i[1]==1:
        p=p+1
    n=n+1
print(n,p)

7470 6524


In [20]:
dataset = dataset.shuffle(20)
dataset = dataset.batch(20)
dataset = dataset.prefetch(2)

testset = testset.shuffle(20)
testset = testset.batch(20)
testset = testset.prefetch(2)

In [13]:
res = tf.keras.applications.EfficientNetB2(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape=(160,160,3),
)
inputs = keras.Input((160, 160, 3), name = "input_1")

features = res(inputs)

x = keras.layers.Flatten()(features)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)

output = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x)

                                               
model = keras.models.Model(inputs=inputs, outputs=output)

31801344/31790344 [==============================] - 0s 0us/step


In [14]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss=loss,
                metrics=metrics)

In [15]:
history = model.fit(dataset, validation_data = testset, epochs=2)

Epoch 1/2


2022-10-09 21:20:58.632706: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-09 21:21:04.536385: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


372/372 [==============================] - 214s 517ms/step - loss: 0.5098 - auc: 0.6229 - val_loss: 0.4508 - val_auc: 0.5000
Epoch 2/2
372/372 [==============================] - 97s 260ms/step - loss: 0.4362 - auc: 0.6469 - val_loss: 0.4681 - val_auc: 0.5395


In [21]:
model.predict(testset.take(2))

Exception ignored in: <generator object test_gen at 0x7fa229c9bb50>
RuntimeError: generator ignored GeneratorExit


array([[0.7296211 ],
       [0.7378318 ],
       [0.73710483],
       [0.7381847 ],
       [0.7379411 ],
       [0.7323993 ],
       [0.73465544],
       [0.7368655 ],
       [0.7377656 ],
       [0.7398045 ],
       [0.74376154],
       [0.7391468 ],
       [0.73591286],
       [0.7437746 ],
       [0.73761755],
       [0.738614  ],
       [0.7402272 ],
       [0.73114026],
       [0.73827964],
       [0.7379648 ],
       [0.73773277],
       [0.73738396],
       [0.739325  ],
       [0.7381236 ],
       [0.738406  ],
       [0.7317348 ],
       [0.7431523 ],
       [0.739329  ],
       [0.7393399 ],
       [0.73494047],
       [0.73909485],
       [0.7371193 ],
       [0.7387639 ],
       [0.73543954],
       [0.73788947],
       [0.74401903],
       [0.7389638 ],
       [0.73703307],
       [0.7376889 ],
       [0.73592293]], dtype=float32)